In [1]:
import time
start_time = time.time()
import os
import sys
import datetime

In [2]:
if len(sys.argv) < 4:
    print('Usage: ./hdf52fits.py subhaloID snap FITS_dir')
    sys.exit(0)

subhalo_id = int(sys.argv[1])
snap = int(sys.argv[2])
img_dir = str(sys.argv[3])

Usage: ./hdf52fits.py subhaloID snap FITS_dir


SystemExit: 0

/opt/anaconda3/envs/astroconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
bands = ['sdss_' + band for band in 'ugriz']
n_pix = 800 # default size

In [4]:
print(bands)

['sdss_u', 'sdss_g', 'sdss_r', 'sdss_i', 'sdss_z']


In [5]:
# permanent storage for FITS files
img_dir = '/u/e/jb2854/Mergers_Project/DF_dir/TNG_FITS_dir/'
# temporary storage for HDF5 files
tmp_dir = '/u/e/jb2854/Mergers_Project/DF_dir/TNG_HDF5_dir/'

In [40]:
snap = 87
subhalo_id = 13297

fitsname = '%s%02d%07d.fits' %(img_dir, snap, subhalo_id)

In [41]:
print(fitsname)

/u/e/jb2854/Mergers_Project/DF_dir/TNG_FITS_dir/870013297.fits


In [42]:
print('Writing final output to %s' %fitsname)
print('Time: ' + str(datetime.timedelta(seconds=(time.time() - start_time))))
sys.stdout.flush()

Writing final output to /u/e/jb2854/Mergers_Project/DF_dir/TNG_FITS_dir/870013297.fits
Time: 0:10:49.831084


In [43]:
import numpy as np
from astropy.table import Table
from astropy.io import fits
from astropy.cosmology import Planck15
import pandas as pd
sys.path.append('/u/e/jb2854/Mergers_Project')
from illustris_python import illustris_python as il

import h5py

In [44]:
api_key = '5f3541d5334553d9d2b98de938189866'
url_prefix = 'https://www.tng-project.org/api/TNG100-1/snapshots/%d/subhalos/%d/' %(snap, subhalo_id)

In [45]:
basePath = '/x/Physics/AstroPhysics/Shared-New/DATA/IllustrisTNG/TNG100-1/output'

In [46]:
table_dir = '/u/e/jb2854/Mergers_Project/DF_dir/'

In [47]:
merger_table_name = 'master_out.parquet'

In [48]:
merger_table = pd.read_parquet(table_dir + merger_table_name)

In [49]:
idx = np.where(np.logical_and(merger_table['DesSFID'] == subhalo_id,
                              merger_table['DesSnap'] == snap))[0]

In [50]:
idx

array([], dtype=int64)

In [51]:
header = fits.Header()
header['HIERARCH subhaloID'] = subhalo_id
header['HIERARCH snapshot'] = snap

In [52]:
header

HIERARCH subhaloID = 13297                                                      
HIERARCH snapshot =                   87                                        

In [53]:
time_table = pd.read_csv(table_dir + 'time_table_TNG_100.csv')

In [54]:
header['HIERARCH redshift'] = float(time_table['Redshift'][snap])
header['HIERARCH lookback_time'] = float(time_table['Lookback'][snap])

In [55]:
header

HIERARCH subhaloID = 13297                                                      
HIERARCH snapshot =                   87                                        
HIERARCH redshift =                 0.15                                        
HIERARCH lookback_time = 1.979                                                  

In [76]:
img_params = {'partType' : 'stars',
              'partField': 'stellarBandObsFrame-', # 'stellarBand-' for RF
              'size'     : 4, #20, #2,
              'sizeType' : 'rHalfMassStars', #'arcmin', #'rVirial',
              'method'   : 'sphMap',
              'nPixels'  : '%d,%d' %(n_pix, n_pix),
              'axes'     : '0,1',  # I think all the arguments below here are
              'rasterPx' : 1100,   # not necessary for the HDF5 files but
              'plotStyle': 'edged',# they don't do any HARM
              'labelZ'   : 'False',
              'labelSim' : 'False',
              'labelHalo': 'False',
              'title'    : 'False',
              'colorbars': 'False',
              'scalebar' : 'False'
             }

In [77]:
for pname in img_params.keys():
    header['HIERARCH ' + pname] = img_params[pname]

In [78]:
header

HIERARCH subhaloID = 13297                                                      
HIERARCH snapshot =                   87                                        
HIERARCH redshift =                 0.15                                        
HIERARCH lookback_time = 1.979                                                  
HIERARCH partType = 'stars   '                                                  
HIERARCH partField = 'stellarBandObsFrame-'                                     
HIERARCH size =                    4                                            
HIERARCH sizeType = 'rHalfMassStars'                                            
HIERARCH method = 'sphMap  '                                                    
HIERARCH nPixels = '800,800 '                                                   
HIERARCH axes = '0,1     '                                                      
HIERARCH rasterPx =                 1100                                        
HIERARCH plotStyle = 'edged 

In [79]:
for i, band in enumerate(bands):
    header['HIERARCH partField_%d' %i] = (header['partField'] + band)

In [80]:
header

HIERARCH subhaloID = 13297                                                      
HIERARCH snapshot =                   87                                        
HIERARCH redshift =                 0.15                                        
HIERARCH lookback_time = 1.979                                                  
HIERARCH partType = 'stars   '                                                  
HIERARCH partField = 'stellarBandObsFrame-'                                     
HIERARCH size =                    4                                            
HIERARCH sizeType = 'rHalfMassStars'                                            
HIERARCH method = 'sphMap  '                                                    
HIERARCH nPixels = '800,800 '                                                   
HIERARCH axes = '0,1     '                                                      
HIERARCH rasterPx =                 1100                                        
HIERARCH plotStyle = 'edged 

In [81]:
if snap == 99:
    redshift = time_table['Redshift'][98]
else:
    redshift = header['Redshift']

In [82]:
subhalo_data = il.groupcat.loadSubhalos(basePath, snap,
                                        fields=['SubhaloMassType',
                                                'SubhaloHalfmassRadType'])

In [83]:
header['HIERARCH stellar_mass'] = (float(subhalo_data['SubhaloMassType'][subhalo_id,
                                                                         il.util.partTypeNum('stellar')]),
                                   '1e10 Msun/h')

In [84]:
header['HIERARCH stellar_half_mass_radius'] = (float(subhalo_data['SubhaloHalfmassRadType'][subhalo_id,
                                                                                            il.util.partTypeNum('stellar')]),
                                                     'ckpc/h')

In [85]:
header['HIERARCH stellar_mass_ratio'] = float(subhalo_data['SubhaloMassType'][subhalo_id,
                                                                              il.util.partTypeNum('stellar')]
                                              / subhalo_data['SubhaloMassType'][subhalo_id,
                                                                               il.util.partTypeNum('dm')])

In [86]:
if snap == 99:
    redshift = time_table['Redshift'][98]
else:
    redshift = header['Redshift']

In [87]:
header['HIERARCH imsize'] = (header['stellar_half_mass_radius'] *
                             img_params['size']/(Planck15.H(0).value/100) *
                             Planck15.arcsec_per_kpc_comoving(redshift).value,
                             'arcsec')

In [88]:
phdu = fits.PrimaryHDU(header=header)
hdul = fits.HDUList([phdu])

In [89]:
hdul

In [90]:
filenames = []

In [91]:
part_field_prefix = img_params['partField']

In [92]:
for axes in ['0,1', '0,2', '1,2']:
    img = []
    for band in bands:
        fname = '%s%d-%d-%s-%s.hdf5' %(tmp_dir, snap, subhalo_id, band, axes)
        filenames.append(fname)
        
        # load the hdf5 file if it exists
        try:
            with h5py.File(fname, 'r') as tmpfile:
                img.append(np.array(tmpfile['grid']))
            print('Opening %s' %fname)
            sys.stdout.flush()

        # if the file does not exist or is corrupted, download it
        except (OSError, KeyError) as e:
            print('Downloading %s from tng-project.org' %fname)
            sys.stdout.flush()
            
            # build filename of download with image params and appropriate band
            dname = 'vis.hdf5?'
            img_params['partField'] = part_field_prefix + band
            for pname in img_params.keys():
                dname = dname + '%s=%s&' %(pname, img_params[pname])
            # dname = dname + 'rotation=%s' %rotation
            #dname = dname + 'axes=%s' %axes
            
            # do the download & open the result
            print('wget --retry-connrefused -O %s --header="API-Key: %s" "%s%s"'
                  %(fname, api_key, url_prefix, dname))
            osanswer = os.system('wget --retry-connrefused -O %s --header="API-Key: %s" "%s%s"'
                                 %(fname, api_key, url_prefix, dname))
            
            try:
                with h5py.File(fname, 'r') as tmpfile:
                    img.append(np.array(tmpfile['grid']))
            except (OSError, KeyError) as e:
                pass
    hdul.append(fits.ImageHDU(data=np.array(img),
                              header=fits.Header([('axes', axes)])))

wget --retry-connrefused -O /u/e/jb2854/Mergers_Project/DF_dir/TNG_HDF5_dir/87-13297-sdss_u-0,1.hdf5 --header="API-Key: 5f3541d5334553d9d2b98de938189866" "https://www.tng-project.org/api/TNG100-1/snapshots/87/subhalos/13297/vis.hdf5?partType=stars&partField=stellarBandObsFrame-sdss_u&size=4&sizeType=rHalfMassStars&method=sphMap&nPixels=800,800&axes=0,1&rasterPx=1100&plotStyle=edged&labelZ=False&labelSim=False&labelHalo=False&title=False&colorbars=False&scalebar=False&"


--2022-10-20 11:03:50--  https://www.tng-project.org/api/TNG100-1/snapshots/87/subhalos/13297/vis.hdf5?partType=stars&partField=stellarBandObsFrame-sdss_u&size=4&sizeType=rHalfMassStars&method=sphMap&nPixels=800,800&axes=0,1&rasterPx=1100&plotStyle=edged&labelZ=False&labelSim=False&labelHalo=False&title=False&colorbars=False&scalebar=False&
Resolving wwwproxy.bath.ac.uk (wwwproxy.bath.ac.uk)... 138.38.44.101
Connecting to wwwproxy.bath.ac.uk (wwwproxy.bath.ac.uk)|138.38.44.101|:3128... connected.
Proxy request sent, awaiting response... 302 Found
Location: https://data-eu.tng-project.org/vis/subhalo/TNG100-1/87/13297/hdf5/?partType=stars&partField=stellarBandObsFrame-sdss_u&size=4&sizeType=rHalfMassStars&method=sphMap&nPixels=800%2C800&axes=0%2C1&rasterPx=1100&plotStyle=edged&labelZ=False&labelSim=False&labelHalo=False&title=False&colorbars=False&scalebar=False&token=23752c8959870710d02e [following]
--2022-10-20 11:03:50--  https://data-eu.tng-project.org/vis/subhalo/TNG100-1/87/13297/

...... 56% 1.73M 0s
  1450K .......... .......... .......... .......... .......... 58%  103M 0s
  1500K .......... .......... .......... .......... .......... 60% 95.7M 0s
  1550K .......... .......... .......... .......... .......... 62% 99.5M 0s
  1600K .......... .......... .......... .......... .......... 64% 94.9M 0s
  1650K .......... .......... .......... .......... .......... 66% 80.6M 0s
  1700K .......... .......... .......... .......... .......... 68%  134M 0s
  1750K .......... .......... .......... .......... .......... 70%  123M 0s
  1800K .......... .......... .......... .......... .......... 72%  127M 0s
  1850K .......... .......... .......... .......... .......... 74%  127M 0s
  1900K .......... .......... .......... .......... .......... 76%  129M 0s
  1950K .......... .......... .......... .......... .......... 78%  140M 0s
  2000K .......... .......... .......... .......... .......... 80%  121M 0s
  2050K .......... .......... .......... .......... .......... 82%  

wget --retry-connrefused -O /u/e/jb2854/Mergers_Project/DF_dir/TNG_HDF5_dir/87-13297-sdss_g-0,1.hdf5 --header="API-Key: 5f3541d5334553d9d2b98de938189866" "https://www.tng-project.org/api/TNG100-1/snapshots/87/subhalos/13297/vis.hdf5?partType=stars&partField=stellarBandObsFrame-sdss_g&size=4&sizeType=rHalfMassStars&method=sphMap&nPixels=800,800&axes=0,1&rasterPx=1100&plotStyle=edged&labelZ=False&labelSim=False&labelHalo=False&title=False&colorbars=False&scalebar=False&"


302 Found
Location: https://data-eu.tng-project.org/vis/subhalo/TNG100-1/87/13297/hdf5/?partType=stars&partField=stellarBandObsFrame-sdss_g&size=4&sizeType=rHalfMassStars&method=sphMap&nPixels=800%2C800&axes=0%2C1&rasterPx=1100&plotStyle=edged&labelZ=False&labelSim=False&labelHalo=False&title=False&colorbars=False&scalebar=False&token=adfa7c4bfd67a45078b8 [following]
--2022-10-20 11:04:35--  https://data-eu.tng-project.org/vis/subhalo/TNG100-1/87/13297/hdf5/?partType=stars&partField=stellarBandObsFrame-sdss_g&size=4&sizeType=rHalfMassStars&method=sphMap&nPixels=800%2C800&axes=0%2C1&rasterPx=1100&plotStyle=edged&labelZ=False&labelSim=False&labelHalo=False&title=False&colorbars=False&scalebar=False&token=adfa7c4bfd67a45078b8
Connecting to wwwproxy.bath.ac.uk (wwwproxy.bath.ac.uk)|138.38.44.101|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 2621440 (2.5M) [application/octet-stream]
Saving to: ‘/u/e/jb2854/Mergers_Project/DF_dir/TNG_HDF5_dir/87-13297-sdss_g-0,1

...

wget --retry-connrefused -O /u/e/jb2854/Mergers_Project/DF_dir/TNG_HDF5_dir/87-13297-sdss_i-1,2.hdf5 --header="API-Key: 5f3541d5334553d9d2b98de938189866" "https://www.tng-project.org/api/TNG100-1/snapshots/87/subhalos/13297/vis.hdf5?partType=stars&partField=stellarBandObsFrame-sdss_i&size=4&sizeType=rHalfMassStars&method=sphMap&nPixels=800,800&axes=0,1&rasterPx=1100&plotStyle=edged&labelZ=False&labelSim=False&labelHalo=False&title=False&colorbars=False&scalebar=False&"


302 Found
Location: https://data-eu.tng-project.org/vis/subhalo/TNG100-1/87/13297/hdf5/?partType=stars&partField=stellarBandObsFrame-sdss_i&size=4&sizeType=rHalfMassStars&method=sphMap&nPixels=800%2C800&axes=0%2C1&rasterPx=1100&plotStyle=edged&labelZ=False&labelSim=False&labelHalo=False&title=False&colorbars=False&scalebar=False&token=76187dadc669568a4d4f [following]
--2022-10-20 11:07:29--  https://data-eu.tng-project.org/vis/subhalo/TNG100-1/87/13297/hdf5/?partType=stars&partField=stellarBandObsFrame-sdss_i&size=4&sizeType=rHalfMassStars&method=sphMap&nPixels=800%2C800&axes=0%2C1&rasterPx=1100&plotStyle=edged&labelZ=False&labelSim=False&labelHalo=False&title=False&colorbars=False&scalebar=False&token=76187dadc669568a4d4f
Connecting to wwwproxy.bath.ac.uk (wwwproxy.bath.ac.uk)|138.38.44.101|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 2621440 (2.5M) [application/octet-stream]
Saving to: ‘/u/e/jb2854/Mergers_Project/DF_dir/TNG_HDF5_dir/87-13297-sdss_i-1,2

 .......... .......... .......... .......... 70% 2.10M 0s
  1800K .......... .......... .......... .......... .......... 72%  174M 0s
  1850K .......... .......... .......... .......... .......... 74%  170M 0s
  1900K .......... .......... .......... .......... .......... 76%  194M 0s
  1950K .......... .......... .......... .......... .......... 78%  221M 0s
  2000K .......... .......... .......... .......... .......... 80%  206M 0s
  2050K .......... .......... .......... .......... .......... 82%  206M 0s
  2100K .......... .......... .......... .......... .......... 83%  235M 0s
  2150K .......... .......... .......... .......... .......... 85%  165M 0s
  2200K .......... .......... .......... .......... .......... 87%  230M 0s
  2250K .......... .......... .......... .......... .......... 89%  207M 0s
  2300K .......... .......... .......... .......... .......... 91%  249M 0s
  2350K .......... .......... .......... .......... .......... 93%  203M 0s
  2400K .......... .......... 

wget --retry-connrefused -O /u/e/jb2854/Mergers_Project/DF_dir/TNG_HDF5_dir/87-13297-sdss_z-1,2.hdf5 --header="API-Key: 5f3541d5334553d9d2b98de938189866" "https://www.tng-project.org/api/TNG100-1/snapshots/87/subhalos/13297/vis.hdf5?partType=stars&partField=stellarBandObsFrame-sdss_z&size=4&sizeType=rHalfMassStars&method=sphMap&nPixels=800,800&axes=0,1&rasterPx=1100&plotStyle=edged&labelZ=False&labelSim=False&labelHalo=False&title=False&colorbars=False&scalebar=False&"


302 Found
Location: https://data-eu.tng-project.org/vis/subhalo/TNG100-1/87/13297/hdf5/?partType=stars&partField=stellarBandObsFrame-sdss_z&size=4&sizeType=rHalfMassStars&method=sphMap&nPixels=800%2C800&axes=0%2C1&rasterPx=1100&plotStyle=edged&labelZ=False&labelSim=False&labelHalo=False&title=False&colorbars=False&scalebar=False&token=fb8c717ce15885af2a86 [following]
--2022-10-20 11:07:29--  https://data-eu.tng-project.org/vis/subhalo/TNG100-1/87/13297/hdf5/?partType=stars&partField=stellarBandObsFrame-sdss_z&size=4&sizeType=rHalfMassStars&method=sphMap&nPixels=800%2C800&axes=0%2C1&rasterPx=1100&plotStyle=edged&labelZ=False&labelSim=False&labelHalo=False&title=False&colorbars=False&scalebar=False&token=fb8c717ce15885af2a86
Connecting to wwwproxy.bath.ac.uk (wwwproxy.bath.ac.uk)|138.38.44.101|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 2621440 (2.5M) [application/octet-stream]
Saving to: ‘/u/e/jb2854/Mergers_Project/DF_dir/TNG_HDF5_dir/87-13297-sdss_z-1,2

In [93]:
# finish setting up FITS file and write
hdul.writeto(fitsname, overwrite=True)

In [75]:
for fname in filenames:
    os.system('rm ' + fname)